# Imports

In [57]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import math

import matplotlib.pyplot as plt
import IPython.display as ipd

sns.set(style="ticks", context="talk", rc={'figure.figsize':(20,10), 'figure.dpi':300})
plt.style.use("dark_background")

In [59]:
path = 'data/original/'
sample_path = 'data/samples/'
all_classes = os.listdir(path)
classes = ['blender',
            'cooking',
            'door_knock',
            'eating',
            'footsteps',
            'human_voice',
            'music',
            'open_faucet',
            'other',
            'take_shower',
            'toilet_flush']

# Make cuts in audios

In [60]:
dict_cut = {}
for c in classes:
    dict_cut[c] = []
    
dict_cut['blender'] = [(11, 97)]
dict_cut['cooking'] = [(10, 415), (10, 190)]
dict_cut['door_knock'] = [(10, 50), (10, 110), (10, 110), (10, 100)]
dict_cut['eating'] = [(10, 490), (5, 365)]
dict_cut['footsteps'] = [(10, 40), (10, 40), (10, 40), (15, 85), (10, 100), (5, 100)]
dict_cut['human_voice'] = [(10, 120), (0, 50), (20, 150), (0, 40)]
dict_cut['music'] = [(10, 180), (10, 190), (10, 200), (0, 200)]
dict_cut['open_faucet'] = [(10, 65), (20, 180)]
dict_cut['take_shower'] = [(20, 420), (30, 340)]
dict_cut['toilet_flush'] = [(10, 20), (10, 20), (10, 20), (10, 20), (10,20), (10, 20), (10,20)]

dict_cut['other'] = [(0, 120), (0, 60), (10, 100)]

# Cut audio and make samples of 5 seconds

In [68]:
data = {'file': [], 'class': [], 'procedence_file': []}
segment_secs = 5000
channels = [str(i) for i in range(8)]

for c in all_classes:
    i = 0
    j = 0
    for cut in dict_cut[c]:
        i += 1
        start = cut[0]
        end = cut[1]
        t1 = start * 1000
        t2 = end * 1000
        
        for channel in channels:
            for k in range(math.floor((t2-t1)/segment_secs)):
                newAudio = AudioSegment.from_wav(path+c+'/0'+str(i)+'/channel_'+channel+'.wav')
                newAudio = newAudio[t1 + k*segment_secs:t1 + (k + 1)*segment_secs]
                output_path = sample_path+c+'_sample_'+str(j)+'.wav'
                newAudio.export(output_path, format="wav")
                data['file'].append(output_path)
                data['class'].append(c)
                data['procedence_file'].append('0'+str(i))
                j += 1
                
df = pd.DataFrame(data)
df.to_csv('home_sound.csv', index=False)

# Cut samples for test data

In [137]:
data = {'file': [], 'class': [], 'procedence_file': []}
segment_secs = 5000
overlapping = 0.5
cuts = [(10, 330)]
data_class = [[(10, 25, 'cooking'), (25, 70, 'other'), (70, 80, 'human_voice'), (80, 100, 'other'), 
              (100, 105, 'footsteps'), (105, 110, 'other'), (110, 125, 'footsteps'), 
              (125, 182.5, 'eating'), (182.5, 207.5, 'human_voice'), (207.5, 297.5, 'eating'), 
              (297.5, 305, 'other'), (305, 330, 'eating')]]
i = 0
for cut in cuts:
    j = 0
    i += 1
    start = cut[0]
    end = cut[1]
    t1 = start * 1000
    t2 = end * 1000
    l = 0
    for k in range(math.floor((((t2-t1)/segment_secs))/overlapping)-1):
        newAudio = AudioSegment.from_wav(path+'test/0'+str(i)+'/channel_0.wav')
        s = t1 + k*segment_secs*overlapping
        e = s + segment_secs
        #print(data_class[i-1])
        if s/1000 >= data_class[i-1][l][0] and e/1000 <= data_class[i-1][l][1]:
            data['class'].append(data_class[i-1][l][2])
        else:
            l += 1
            data['class'].append(data_class[i-1][l][2])
        #print(s/1000, '-', e/1000, data['class'][-1])
        newAudio = newAudio[s:e]
        output_path = 'data/test/0'+str(i)+'/test_sample_'+str(j)+'.wav'
        newAudio.export(output_path, format="wav")
        data['file'].append(output_path)
        data['procedence_file'].append('0'+str(i))
        j += 1
        
df = pd.DataFrame(data)
df.to_csv('home_sound_test.csv', index=False)